# Feinschmecker

In [57]:
from exceptiongroup import catch
from owlready2 import *
import json


### Factory creations

In [58]:
NAMESPACE = "https://jaron@sprute.com/uni/actionable-knowledge-representation/feinschmecker"
onto = get_ontology(NAMESPACE)


def ThingFactory(name, BaseClass=Thing) -> type[Thing]:
    with onto:
        return type[Thing](name, (BaseClass,), {})


def RelationFactory(name, domain: list[ThingClass] = None, range=None) -> type[ObjectProperty]:
    if domain is None:
        domain = [Thing]
    if range is None:
        range = [Thing]
    with onto:
        return type[ObjectProperty](name, (ObjectProperty,), {
            "domain": domain,
            "range": range,
        })


def DataFactory(name, domain: list[ThingClass] = None, range=None) -> type[DataProperty]:
    if domain is None:
        domain = [Thing]
    if range is None:
        range = [str]
    with onto:
        return type[DataProperty](name, (DataProperty,), {
            "domain": domain,
            "range": range,
        })


def makeInverse(first: ObjectProperty, second: ObjectProperty) -> None:
    if first is None or second is None:
        raise TypeError("There is no inverse of no element: first:", str(first), "second:", str(second))
    with onto:
        first.inverse_property = second
        second.inverse_property = first


### Class creations

In [59]:
onto.metadata.comment.append("This project is about recipes that are used for meal preparations found in the web.")

Recipe = ThingFactory("Recipe")
Recipe.comment = "A list of steps needed to prepare a meal, along with time of preparation, amount of calories, amounts of macroelements and information about it being vegetarian or vegan"
Ingredient = ThingFactory("Ingredient")
Ingredient.comment = "The ingredient type or name used within a recipe."
IngredientWithAmount = ThingFactory("RecipeIngredientRelation")
IngredientWithAmount.comment = "Technically required class in order to save the amount of a certain ingredient required for a certain recipe"
Author = ThingFactory("Author")
Author.comment = "The name (or username) of the author of the recipe written on a certain website."
Source = ThingFactory("Source")
Source.comment = "The website name including URL on which the recipe is found."
Time = ThingFactory("Time")
Time.comment = "The time required to finish the preparations of a meal according to a specific recipe. The unit is minutes."

Nutrients = ThingFactory("Nutrients")
Nutrients.comment = "The nutrients one can expect to find within the meal according to a certain recipe, divided into Proteins, Fats, Carbohydrates and optionally Calories."
Calories = ThingFactory("Calories", BaseClass=Nutrients)
Calories.comment = "The calories one can expect to find within the meal according to a certain recipe. The unit is kilocalorie."
Protein = ThingFactory("Protein", BaseClass=Nutrients)
Protein.comment = "The protein one can expect to find within the meal according to a certain recipe. The unit is gram."
Fat = ThingFactory("Fat", BaseClass=Nutrients)
Fat.comment = "The fat one can expect to find within the meal according to a certain recipe. The unit is gram."
Carbohydrates = ThingFactory("Carbohydrates", BaseClass=Nutrients)
Carbohydrates.comment = "The carbohydrates one can expect to find within the meal according to a certain recipe. The unit is gram."

### Relation creations

In [60]:
# Descends according to the class creations above

# Recipe
has_instructions = DataFactory("has_instructions", domain=[Recipe], range=[str])
has_instructions.comment = "A description of the instructions one has to follow according to the recipe in form of a text."

has_ingredient = RelationFactory("has_ingredient", domain=[Recipe], range=[IngredientWithAmount])
has_ingredient.comment = "A link from the recipe to one of the ingredients including their amount required for it."
authored_by = RelationFactory("authored_by", domain=[Recipe], range=[Author])
authored_by.comment = "A link from the recipe to its author."
requires_time = RelationFactory("requires_time", domain=[Recipe], range=[Time])
requires_time.comment = "A link from the recipe to the time required in order to create the in the recipe specified meal."

is_vegan = DataFactory("is_vegan", domain=[Recipe], range=[bool])
is_vegan.comment = "A description of whether the recipe is vegan or not."
is_vegetarian = DataFactory("is_vegetarian", domain=[Recipe], range=[bool])
is_vegetarian.comment = "A description of whether the recipe is vegetarian or not."
has_difficulty = DataFactory("has_difficulty", domain=[Recipe], range=[float])
has_difficulty.comment = "A description of the recipes difficulty in form of a floating point number from 1 to 3 where 1 represents easy, 2 moderately hard and 3 hard."

has_calories = RelationFactory("has_calories", domain=[Recipe], range=[Calories])
has_calories.comment = "A link from the recipe to the amount of calories a meal created according to it approximately has"
has_protein = RelationFactory("has_protein", domain=[Recipe], range=[Protein])
has_protein.comment = "A link from the recipe to the amount of protein a meal created according to it approximately has"
has_fat = RelationFactory("has_fat", domain=[Recipe], range=[Fat])
has_fat.comment = "A link from the recipe to the amount of fat a meal created according to it approximately has"
has_carbohydrates = RelationFactory("has_carbohydrates", domain=[Recipe], range=[Carbohydrates])
has_carbohydrates.comment = "A link from the recipe to the amount of carbohydrates a meal created according to it approximately has"

has_link = DataFactory("has_link", domain=[Recipe], range=[str]) # RDF.HTML is not supported, see https://owlready2.readthedocs.io/en/latest/properties.html#data-property 
has_link.comment = "A description of the URL the recipe was found on. Since URLs are not supported by owlready2 in form of a text."

# Ingredient
is_ingredient_of = RelationFactory("is_ingredient_of", domain=[Ingredient], range=[IngredientWithAmount])
is_ingredient_of.comment = "A link from the ingredient to the ingredient with amount which is required for at least one recipe."

# IngredientWithAmount
used_for = RelationFactory("used_for", domain=[IngredientWithAmount], range=[Recipe])
used_for.comment = "A link from the ingredient with amount to the recipe which requires that specific amount of the ingredient."
type_of_ingredient = RelationFactory("type_of_ingredient", domain=[IngredientWithAmount], range=[Ingredient])
type_of_ingredient.comment = "A link from the ingredient with amount to the ingredient it has the amount of."
amount_of_ingredient = DataFactory("amount_of_ingredient", domain=[IngredientWithAmount], range=[float])
amount_of_ingredient.comment = "A description of the numeric quantity the ingredient with amount has the ingredient as a floating point number."
unit_of_ingredient = DataFactory("unit_of_ingredient", domain=[IngredientWithAmount], range=[str])
unit_of_ingredient.comment = "A description of the unit the ingredient with amount has the ingredient as a text."

# Author
authored = RelationFactory("authored", domain=[Author], range=[Recipe])
authored.comment = "A link from the author to the recipe the author authored."
is_author_of = RelationFactory("is_author_of", domain=[Author], range=[Source])
is_author_of.comment = "A link from the author in form of a user to the source/website he is part of."

# Source
has_author = RelationFactory("has_author", domain=[Source], range=[Author])
has_author.comment = "A link from the source/website to the author in form of a user using this website."
is_website = DataFactory("is_website", domain=[Source], range=[str]) # RDF.HTML is not supported, see https://owlready2.readthedocs.io/en/latest/properties.html#data-property
is_website.comment = "A description of the URL the source/website uses. Since URLs are not supported by owlready2 in form of a text."

# Time
time_required_by = RelationFactory("time_required", domain=[Time], range=[Recipe])
time_required_by.comment = "A link from the time to the recipe which requires the time to complete its meal."
amount_of_time = DataFactory("amount_of_time", domain=[Time], range=[int])
amount_of_time.comment = "A description of the duration of the time in minutes as a number."

# Nutrients
calories_of = RelationFactory("calories_of", domain=[Calories], range=[Recipe])
calories_of.comment = "A link from the calories to the recipe whose meal approximately has the amount of calories of."
amount_of_calories = DataFactory("amount_of_calories", domain=[Calories], range=[float])
amount_of_calories.comment = "A description of the amount the calories has in kilocalorie as a floating point number."
protein_of = RelationFactory("protein_of", domain=[Protein], range=[Recipe])
protein_of.comment = "A link from the protein to the recipe whose meal approximately has the amount of protein of."
amount_of_protein = DataFactory("amount_of_protein", domain=[Protein], range=[float])
amount_of_protein.comment = "A description of the amount the protein has in gram as a floating point number."
fat_of = RelationFactory("fat_of", domain=[Fat], range=[Recipe])
fat_of.comment = "A link from the fat to the recipe whose meal approximately has the amount of fat of."
amount_of_fat = DataFactory("amount_of_fat", domain=[Fat], range=[float])
amount_of_fat.comment = "A description of the amount the fat has in gram as a floating point number."
carbohydrates_of = RelationFactory("carbohydrates_of", domain=[Carbohydrates], range=[Recipe])
carbohydrates_of.comment = "A link from the carbohydrates to the recipe whose meal approximately has the amount of carbohydrates of."
amount_of_carbohydrates = DataFactory("amount_of_carbohydrates", domain=[Carbohydrates], range=[float])
amount_of_carbohydrates.comment = "A description of the amount the carbohydrates has in gram as a floating point number."

### Interrelational properties

In [61]:
# Inverses
makeInverse(has_ingredient, used_for)
makeInverse(is_ingredient_of, type_of_ingredient)
makeInverse(authored_by, authored)
makeInverse(is_author_of, has_author)
makeInverse(requires_time, time_required_by)
makeInverse(has_calories, calories_of)
makeInverse(has_protein, protein_of)
makeInverse(has_fat, fat_of)
makeInverse(has_carbohydrates, carbohydrates_of)

# Limitations
with onto:
    # Recipe
    Recipe.is_a.append(has_instructions.exactly(1, str))
    Recipe.is_a.append(has_ingredient.some(IngredientWithAmount))
    Recipe.is_a.append(authored_by.exactly(1, Author))
    Recipe.is_a.append(requires_time.exactly(1, Time))
    Recipe.is_a.append(is_vegan.max(1, bool))
    Recipe.is_a.append(is_vegetarian.max(1, bool))
    Recipe.is_a.append(has_difficulty.max(1, int))  # Debatable
    Recipe.is_a.append(has_calories.max(1, Calories)) 
    Recipe.is_a.append(has_protein.exactly(1, Protein))
    Recipe.is_a.append(has_fat.exactly(1, Fat))
    Recipe.is_a.append(has_carbohydrates.exactly(1, Carbohydrates))
    Recipe.is_a.append(has_link.exactly(1, str))

    # Ingredient
    
    # RecipeIngredientRelation
    IngredientWithAmount.is_a.append(used_for.some(Recipe))
    IngredientWithAmount.is_a.append(type_of_ingredient.exactly(1, Ingredient))
    IngredientWithAmount.is_a.append(amount_of_ingredient.exactly(1, int))
    IngredientWithAmount.is_a.append(unit_of_ingredient.exactly(1, str))

    # Source
    Source.is_a.append(is_website.exactly(1, str))
    
    # Time
    Time.is_a.append(amount_of_time.exactly(1, int))

    # Calories
    Calories.is_a.append(amount_of_calories.exactly(1, float))
    Protein.is_a.append(amount_of_protein.exactly(1, float))
    Fat.is_a.append(amount_of_fat.exactly(1, float))
    Carbohydrates.is_a.append(amount_of_carbohydrates.exactly(1, float))


# Disjointness
with onto:
    AllDisjoint([Recipe, Ingredient, IngredientWithAmount, Author, Source, Time, Nutrients])
    AllDisjoint([Calories, Protein, Fat, Carbohydrates])

### Individuals from JSON file

In [62]:
# Creates individual or returns it if it exists(existing -> bool = 1) with a fitting type
def createIndividual(name, BaseClass, unique=False) -> (Thing, bool):
    if onto[name] is None:
        return BaseClass(name), False
    if type(onto[name]) != BaseClass or unique:
        raise TypeError("Individual " + name + " already exists:\nExisting: " + str(type(onto[name])) + "\nRequested: " + str(BaseClass))
    return onto[name], True


with open("recipes.json", "r") as json_recipe:
    recipes = json.load(json_recipe)


mainSource = ("Example", "https://example.org")
source, _ = createIndividual(mainSource[0], BaseClass=Source, unique=True)
source.is_website.append(mainSource[1])


for json_recipe in recipes:
    recipe, _ = createIndividual(json_recipe["title"], BaseClass=Recipe, unique=True)
    recipe.has_instructions.append("Just do it correctly ;D")

    # IngredientWithAmount
    for extendedIngredient in json_recipe["ingredients"]:
        ingredientWithAmount, existed = createIndividual(extendedIngredient["id"], BaseClass=IngredientWithAmount)
        if existed:
            recipe.has_ingredient.append(ingredientWithAmount)
            continue
        if extendedIngredient["quantity"] is not None:
            ingredientWithAmount.amount_of_ingredient.append(extendedIngredient["quantity"])
        else:
            ingredientWithAmount.amount_of_ingredient.append(1)
        ingredientWithAmount.unit_of_ingredient.append(str(extendedIngredient["unit"]))
        ingredient, _ = createIndividual(extendedIngredient["ingredient"]["id"], BaseClass=Ingredient)
        ingredientWithAmount.type_of_ingredient.append(ingredient)
        recipe.has_ingredient.append(ingredientWithAmount)

    # Author
    author, existed = createIndividual(json_recipe["author"], BaseClass=Author)
    if not existed:
        author.is_author_of.append(source)
    recipe.authored_by.append(author)
    
    # Time
    time, existed = createIndividual("time_" + str(json_recipe["time"][0]), BaseClass=Time)
    if not existed:
        time.amount_of_time.append(json_recipe["time"][0])
    recipe.requires_time.append(time)
    
    recipe.is_vegan.append(json_recipe["vegan"])
    recipe.is_vegetarian.append(json_recipe["vegetarian"])
    if len(recipe.has_ingredient)*3 + time.amount_of_time[0] < 20: # Easy
        recipe.has_difficulty.append(1)
    elif len(recipe.has_ingredient)*3 + time.amount_of_time[0] < 60: # Middle
        recipe.has_difficulty.append(2)
    else: # Difficult
        recipe.has_difficulty.append(3)
    
    # Nutrients
    nutrients = json_recipe["nutrients"]
    calories, existed = createIndividual("calories_" + nutrients["kcal"][0], BaseClass=Calories)
    if not existed:
        calories.amount_of_calories.append(float(nutrients["kcal"][0]))
    recipe.has_calories.append(calories)
    
    protein, existed = createIndividual("protein_" + nutrients["protein"][0], BaseClass=Protein)
    if not existed:
        protein.amount_of_protein.append(float(nutrients["protein"][0]))
    recipe.has_protein.append(protein)
    
    fat, existed = createIndividual("fat_" + nutrients["fat"][0], BaseClass=Fat)
    if not existed:
        fat.amount_of_fat.append(float(nutrients["fat"][0]))
    recipe.has_fat.append(fat)
    
    carbohydrates, existed = createIndividual("carbohydrates_" + nutrients["carbs"][0], BaseClass=Calories)
    if not existed:
        carbohydrates.amount_of_carbohydrates.append(float(nutrients["carbs"][0]))
    recipe.has_carbohydrates.append(carbohydrates)
    
    recipe.has_link.append(json_recipe["source"])



In [63]:
with onto:
    print(list(default_world.inconsistent_classes()))
    print(len(list(map(lambda a: a.name, onto.individuals()))))

print(onto["Spinach, sweet potato & lentil dhal"])
print(type(onto["Spinach, sweet potato & lentil dhal"]))
print(onto["Spinach, sweet potato & lentil dhal"].has_ingredient)



[]
235
feinschmecker.Spinach, sweet potato & lentil dhal
feinschmecker.Recipe
[feinschmecker.sesame_oil_with_amount, feinschmecker.onion_finely_chopped_with_amount, feinschmecker.clove_crushed_with_amount, feinschmecker.-sized_piece_ginger_peeled_and_finely_chopped_with_amount, feinschmecker.chilli_finely_chopped_with_amount, feinschmecker.½_tsp_ground_turmeric_with_amount, feinschmecker.½_tsp_ground_cumin_with_amount, feinschmecker.potatoes_(about_400g/14oz),_cut_into_even_chunks_with_amount, feinschmecker.red_split_lentils_with_amount, feinschmecker.vegetable_stock_with_amount, feinschmecker.bag_of_spinach_with_amount, feinschmecker.onions_sliced_on_the_diagonal,_to_serve_with_amount, feinschmecker.½_small_pack_of_thai_basil_leaves_torn,_to_serve_with_amount]


## Temporary deletion to avoid side effects

In [64]:
onto.save("feinschmecker.rdf")
onto.destroy(update_relation=True, update_is_a=True)